# Imports

In [1]:
import sys
import os
import numpy as np
import torch
import skfmm

# Constants

Dataset directory structure:
```
📂datasets
 ┗ 📂{dataset name}
   ┗ 📂{subset}
     ┗ 📂{map name}
       ┣ 📜{map name}.txt
       ┗ 📂paths
         ┗ 📜{path name}.txt
```


Input directory structure:
```
📂inputs
 ┗ 📂{dataset name}
   ┗ 📂{subset}
     ┗ 📜{processed input item}
```

In [2]:
INPUT_DIR = 'datasets'  # The directory containing the map(s) & paths to be processed
OUTPUT_DIR = 'test_data'       # The directory where this notebook's output will be stored

DATASET = 'test_set'        # dataset to prepare
SUBSET = None       # Subset to prepare (leave as None to prepare all subsets)

# Process Inputs

In [3]:
def convert_path(path_file, map_mat, dest_dir):
    # TODO: check that the path file exists
    if not os.path.isfile(path_file):
        print(f"ERROR: Path file {path_file} does not exist")
        return

    # If the output destination doesn't exist, create the directory
    if not os.path.isdir(dest_dir):
        os.makedirs(dest_dir)

    path = np.loadtxt(path_file)
    path = np.asarray(path, dtype=int).reshape(len(path)//2,2)

    path_mat = np.zeros_like(map_mat[0,:,:])
    end_mat = np.ones_like(map_mat[0,:,:])

    # Make the path continuous
    for i in range(path.shape[0] - 1):
        x = path[i,0]
        x1 = path[i,0]
        x2 = path[i+1,0]

        y = path[i,1]
        y1 = path[i,1]
        y2 = path[i+1,1]

        if (x1 < x2):
            x_dir = 1
        else:
            x_dir = -1

        if (y1 < y2):
            y_dir = 1
        else:
            y_dir = -1

        # Determine y from x
        if x2-x1 != 0:
            m = (y2-y1)/(x2-x1)
            while x != x2:
                y = round(m*(x-x1) + y1)
                path_mat[y,x] = 1
                x += x_dir
        else:
            while x != x2:
                path_mat[y1,x] = 1
                x += x_dir


        x = path[i,0]
        x1 = path[i,0]
        x2 = path[i+1,0]

        y = path[i,1]
        y1 = path[i,1]
        y2 = path[i+1,1]

        # Determine x from y
        if y2-y1 != 0:
            m = (x2-x1)/(y2-y1)
            while y != y2:
                x = round(m*(y-y1) + x1)
                path_mat[y,x] = 1
                y += y_dir
        else:
            while y != y2:
                path_mat[y,x1] = 1
                y += y_dir
        
    path_mat[path[path.shape[0]-1,1], path[path.shape[0]-1,0]] = 1     # Include the last point in the path


    # Create endpoints matrix
    end_mat *= -1
    end_mat[path[0,1], path[0,0]] = 1
    end_mat[path[path.shape[0]-1,1], path[path.shape[0]-1,0]] = 1
    end_mat = skfmm.distance(end_mat, dx=0.1)
    end_mat += 1

    # Combine the three matrices
    path_mat = path_mat[np.newaxis,:,:]
    end_mat = end_mat[np.newaxis,:,:]
    path = np.concatenate((path_mat, end_mat, map_mat), axis=0)
    path = torch.Tensor(path)

    # Save path to file
    num_paths = 0
    path_list = os.scandir(dest_dir)
    for item in path_list:
        num_paths += 1
    file_name = f"path_{num_paths}.pt"

    torch.save(path, os.path.join(dest_dir, file_name))

In [4]:
def convert_map(map_dir, dest_dir):

    if not os.path.isdir(map_dir):
        print(f"ERROR: Map directory '{map_dir}' does not exist")
        return

    map_name = os.path.basename(map_dir)
    map_file = os.path.join(map_dir, f"{map_name}.txt")
    if not os.path.isfile(map_file):
        print(f"ERROR: Missing map file '{map_file}'")
        return

    with open(map_file) as file:
        map_dims = file.readline()
    map_dims = tuple(map(int, map_dims.split(' ')))

    map_mat = np.loadtxt(map_file, skiprows=2).reshape(map_dims)
    # TODO: process map using SDF
    map_mat *= -1
    map_mat[map_mat == 0] = 1
    map_mat = skfmm.distance(map_mat, dx = 0.1)
    map_mat = map_mat[np.newaxis, :, :]

    path_dir = os.path.join(map_dir, 'paths')
    if not os.path.isdir(path_dir):
        print(f"ERROR: Path directory {path_dir} does not exist")
        return
    
    # Convert each path
    paths = os.scandir(path_dir)
    for item in paths:
        convert_path(item.path, map_mat, dest_dir)

In [5]:
def convert_subset(subset_dir, dest_dir):
    
    if not os.path.isdir(subset_dir):  # make sure the directory exists
        print(f"ERROR: subset '{subset_dir}' does not exist")
        return

    # process each map along with its paths
    maps = os.scandir(subset_dir)
    for item in maps:
        obs_map = convert_map(item.path, os.path.join(dest_dir, item.name))

In [6]:
# Open the specified dataset's directory
in_dir = os.path.join(os.getcwd(), INPUT_DIR, DATASET)
out_dir = os.path.join(os.getcwd(), OUTPUT_DIR, DATASET)

if not os.path.isdir(in_dir):
    print(f"Error: dataset {in_dir} does not exist")
    sys.exit()

if SUBSET != None:                  # open specified subset
    in_dir = os.path.join(in_dir, SUBSET)
    out_dir = os.path.join(out_dir, SUBSET)
    convert_subset(in_dir, out_dir)
else:                               # process all subsets in the dataset
    subsets = os.scandir(in_dir)
    for item in subsets:
        convert_subset(item.path, os.path.join(out_dir, item.name))

# Testing this script

In [ ]:
from matplotlib import pyplot

In [ ]:
# Load some example input
# Use pyplot to display the example(s)